# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system

In [1]:
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 2.180641175584646 
 1.7063263884865882
 1.3393760546009583

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [4]:
Alu = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.909126   1.0       0.0
 0.234992  -0.435304  1.0
U factor:
3×3 Array{Float64,2}:
 0.842533   0.610152   0.727956 
 0.0       -0.269668  -0.0064832
 0.0        0.0        0.706734 

In [6]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

#### Coming in 1.0: accessing factorization components

*In 1.0, the syntax for accessing the components of a factorization has changed. To access the lower triangular matrix stored in the factorization object `Alu`, you'll write*

```julia
Alu.L
```

*instead of*

```julia
Alu[:L]
```

In [9]:
Alu.L

3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.909126   1.0       0.0
 0.234992  -0.435304  1.0

In [10]:
Alu.U

3×3 Array{Float64,2}:
 0.842533   0.610152   0.727956 
 0.0       -0.269668  -0.0064832
 0.0        0.0        0.706734 

In [11]:
Alu.P

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [12]:
A\b

3-element Array{Float64,1}:
 1.0               
 1.0000000000000004
 0.9999999999999999

In [13]:
Alu\b

3-element Array{Float64,1}:
 1.0               
 1.0000000000000004
 0.9999999999999999

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [14]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [15]:
Aqr = qr(A) # in 1.0, qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.728989   0.466845  -0.50063 
 -0.662743  -0.664375   0.34551 
 -0.171306   0.583662   0.793721
R factor:
3×3 Array{Float64,2}:
 -1.15576  -0.678372  -1.11584 
  0.0       0.247676   0.418448
  0.0       0.0        0.560949

The matrices `Q` and `R` can be extracted from the QR factorization object via

In [16]:
Aqr.Q # in 1.0, Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.728989   0.466845  -0.50063 
 -0.662743  -0.664375   0.34551 
 -0.171306   0.583662   0.793721

In [17]:
Aqr.R # in 1.0, Aqr.R

3×3 Array{Float64,2}:
 -1.15576  -0.678372  -1.11584 
  0.0       0.247676   0.418448
  0.0       0.0        0.560949

In [22]:
U = Aqr.Q
U'U

3×3 Array{Float64,2}:
  1.0          -2.77556e-17  -1.38778e-16
 -2.77556e-17   1.0           1.66533e-16
 -1.38778e-16   1.66533e-16   1.0        

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [26]:
Asym = A + A'
AsymEig = eigvals(Asym) # in 1.0 eig(Asym)

3-element Array{Float64,1}:
 -0.3936524424297563
  0.8693807055198008
  3.5406514365210273

The values and the vectors can be extracted from the Eigen type by special indexing

In [31]:
AsymEigVec = eigvecs(Asym) # in 1.0, AsymEig.vectors

3×3 Array{Float64,2}:
  0.499037  -0.572588  -0.650465
 -0.853759  -0.196215  -0.482281
  0.148517   0.796017  -0.586771

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [32]:
inv(AsymEig)*Asym

MethodError: MethodError: no method matching inv(::Array{Float64,1})
Closest candidates are:
  inv(!Matched::Complex{Float64}) at complex.jl:396
  inv(!Matched::BigFloat) at mpfr.jl:418
  inv(!Matched::Integer) at int.jl:56
  ...

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [34]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [35]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [36]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

3.0842740234514325

In [37]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [38]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [39]:
@time eigvals(Asym);

  0.124115 seconds (16 allocations: 7.989 MiB)


In [40]:
@time eigvals(Asym_noisy);

  1.137181 seconds (18 allocations: 7.921 MiB, 6.48% gc time)


In [41]:
@time eigvals(Asym_explicit);

  0.125935 seconds (7.62 k allocations: 8.378 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [42]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.951049 seconds (484.49 k allocations: 206.839 MiB, 12.42% gc time)


6.956061458715677

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [43]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [44]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Array{Rational{BigInt},2}:
 7//10  2//5  1//1 
 2//5   1//2  7//10
 1//2   1//2  7//10

In [45]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
 21//10
  8//5 
 17//10

In [46]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [47]:
lu(Arational) # in 1.0, lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
 1//1   0//1   0//1
 4//7   1//1   0//1
 5//7  15//19  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 7//10   2//5     1//1 
 0//1   19//70    9//70
 0//1    0//1   -11//95

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

In [23]:
using LinearAlgebra
A = [140   97   74  168  131; 97  106   89  131   36; 74   89  152  144   71; 168  131  144   54  142; 131   36   71  142   36]
eigvals(A)

5-element Array{Float64,1}:
 -128.49322764802145 
  -55.887784553056875
   42.7521672793189  
   87.16111477514521 
  542.4677301466143  

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [21]:
Λ = eigvals(A)
Diagonal(Λ)

5×5 Diagonal{Float64,Array{Float64,1}}:
 -128.493     ⋅        ⋅        ⋅         ⋅   
     ⋅     -55.8878    ⋅        ⋅         ⋅   
     ⋅        ⋅      42.7522    ⋅         ⋅   
     ⋅        ⋅        ⋅      87.1611     ⋅   
     ⋅        ⋅        ⋅        ⋅      542.468

#### 11.3 
Create a `LowerTriangular` matrix from `A`.

In [24]:
LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback